#### Python Libraries

In [2]:
import pickle as pkl
import networkx as nx
import numpy as np
from tqdm import tqdm
from itertools import chain
import scipy.sparse as ss

#### Own Libraries

In [3]:
from lib import translateGraphs
from lib import save_sparse_csr
from lib import getnodelblarr, getedgelabelarr, hashtodic, assignewlabels_node, assignewlabels_edge
from lib import generateFeatureVectorsfromAlphabet
from lib import my_train_test_split
from lib import calculateLogits
from lib import saveDataToFormattedSubmissionFile

#### Load Data

In [4]:
with open('Data/test_data.pkl', 'rb') as file:
    test_graphs = pkl.load(file)

with open('Data/training_data.pkl', 'rb') as file:
    train_graphs = pkl.load(file)

#### Change labels of test graphs

In [5]:
print('Relabel test_graphs')
test_graphs = translateGraphs(test_graphs)

Relabel test_graphs


100%|██████████| 2000/2000 [00:00<00:00, 3097.76it/s]


#### Generate WL Graph Sequence

In [6]:
def WL(graphs, h=4):
    #Number of hops
    N = len(graphs)# N = len(train_graphs)//1
    
    hashedgraphs = graphs#train_graphs.copy()
    hashedgraphs = hashedgraphs[0:N]
    Alphabet = {}#{f'{i}': i for i in range(49)} #{}
    currentmax = 53

    #Alphabet instances for each graph over all hops
    #Initialise lv - our list of node-labels
    #       and ev - our list of edge-labels
    lv = [[] for _ in range(len(hashedgraphs))]
    el = [[] for _ in range(len(hashedgraphs))]
    for i in range(len(hashedgraphs)):
        #get node labels, edgelabels
        v = nx.get_node_attributes(hashedgraphs[i], 'labels').values()
        e = nx.get_edge_attributes(hashedgraphs[i], 'labels').values()
        #extract from list-of-list
        v = list(chain.from_iterable(v))
        e = list(chain.from_iterable(e))
        #add []+a
        lv[i] = lv[i] + v
        el[i] = el[i] + e

    print(f'Hops = {h}')
    #For each hop
    for _ in range(h):
        #For each graph
        for i in tqdm(range(len(hashedgraphs))):
            #graph to be worked with
            Gi = hashedgraphs[i].copy()
            #simplify: get biggest connected subgraph
            Gi = Gi.subgraph(sorted(nx.connected_components(Gi), key=len, reverse=True)[0])
            
            #For each node
            #Array with columns: node, label, newlabel
            lblarri_v = getnodelblarr(Gi)
            lblarri_e = getedgelabelarr(Gi)
            #Update big alphabet, hash
            Alphabet, lblarri_v, currentmax = hashtodic(Alphabet, lblarri_v, currentmax, node=True)
            #sort and reset index
            lblarri_v = lblarri_v.sort_values(by='node').reset_index(drop=True)  
            #relabel to a different graph
            # print(lblarri_v, lblarri_e, sep='\n\n')
            Gi = assignewlabels_node(Gi, lblarri_v)

            if len(lblarri_e)!=0:
                Alphabet, lblarri_e, currentmax = hashtodic(Alphabet, lblarri_e, currentmax, node=False)
                lblarri_e = lblarri_e.sort_values(by='edge').reset_index(drop=True)
                Gi = assignewlabels_edge(Gi, lblarri_e)
                e = nx.get_edge_attributes(hashedgraphs[i], 'labels').values()
                e = list(chain.from_iterable(e))
                el[i] = el[i] + e

            #assign graph-value
            hashedgraphs[i] = Gi
            #add counts of labels, reuse previous variable
            v = nx.get_node_attributes(hashedgraphs[i], 'labels').values()
            v = list(chain.from_iterable(v))
            lv[i] = lv[i] + v
            
    return hashedgraphs, Alphabet, lv, el

In [6]:
hashedgraphs, Alphabet, lv, ev = WL(train_graphs + test_graphs, h=8)

Hops = 8


100%|██████████| 8000/8000 [07:28<00:00, 17.84it/s]


#### Generate feature vectors from alphabet

In [7]:
#Matrix of feature vectors, will be very sparse.
sM = generateFeatureVectorsfromAlphabet(hashedgraphs, Alphabet, lv)

100%|██████████| 8000/8000 [00:03<00:00, 2539.89it/s]


##### Save Data for easier reloading

In [8]:
save_sparse_csr('Data/WL_allfeatures_e.npz', sM)
save_sparse_csr('Data/WL_trainfeatvec_e.npz', sM[0:len(train_graphs)])
save_sparse_csr('Data/WL_testfeatvec_e.npz', sM[len(train_graphs):len(train_graphs+test_graphs)])

#### Create kernel matrix from feature vectors

In [10]:
K = np.dot(sM, sM.T)
# np.savetxt('WL_Kernel.txt', K)
K.shape

(8000, 8000)

Center dataset: $K^c=(I-U)K(I-U)$

In [7]:
# Calculate
N = K.shape[0]
K = ss.csr_matrix.toarray(K)
U = (1/N) * np.ones((N,N))
I = np.eye(N)
Kc = (I-U) @ K @ (I-U)
# np.savetxt('Data/Kc8000.txt', Kc)

# Or load previously generated
# Kc = np.loadtxt('Data/Kc8000.txt')
print(Kc.shape)

(8000, 8000)


#### Classify

Load data

In [8]:
with open('Data/training_labels.pkl', 'rb') as file:
    labels = pkl.load(file)

WLData = Kc

Project entire dataset (train-validate-test) to PCA

In [9]:
evals, evecs = np.linalg.eigh(Kc)
numpca = 8000
WLData_projected = Kc.dot(evecs[0:numpca].T)
np.savetxt('Data/Kc8000_pcaed.txt', WLData_projected)

In [11]:
# WLData_projected = np.loadtxt('Data/Kc8000_pcaed.txt')
N = 6000

#### Training Classifier

We attempted the use of the SVM classifier from HW2 but the algorithm would not converge. 

In [ ]:
# from lib import my_train_test_split
# WLTrainValid = WLData_projected[0:N]
# WLLabels = labels[0:N]

# from lib import LIN, KernelSVC
# sigma = 1.5
# C=100.
# kernel = LIN().kernel
# linmodel = KernelSVC(C=C, kernel=kernel)

# X_train, X_validate, y_train, y_validate = my_train_test_split(WLTrainValid, WLLabels, test_size=0.2, random_state=1)
# #Does not converge
# linmodel.fit(X_train, y_train)
# # linmodel.predict(X_validate)

We also wrote our own kernel logistic regression as in the class slides, but this gave a different error in the exponents. As this was not fixed in time, I opt to use an sklearn library in the hopes of at least showcasing the effectiveness of the WL-algorithm.

In [63]:
#Choose subset
rows = N
cols = 8000
WLTrainValid = WLData_projected[0:rows, 0:cols]
WLLabels = labels[0:rows]

#Split data
X_train, X_validate, y_train, y_validate = my_train_test_split(WLTrainValid, WLLabels, test_size=0.2, random_state=1)

#Logistic regression
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(solver='newton-cholesky', verbose=False, penalty='l2', C=200, random_state=1)

# Fit on training data
lr.fit(X_train, y_train)


In [ ]:
#Predict on validation data
pred = lr.predict(X_validate[:, 0:1000])

#AUC metric
from sklearn import metrics
fpr, tpr, thresholds = metrics.roc_curve(y_validate, pred, pos_label=1)
print(f'AUC: {metrics.auc(fpr, tpr)}')

AUC: 0.7451557973851092


Predict on Test Data

In [36]:
X_test = WLData_projected[6000:8000, 0:cols]
pred_test = lr.predict_proba(X_test)
logit = calculateLogits(pred_test)

Logit range:
 [-inf, 36.04365338911715]


c:\Users\norti\OneDrive - Université Paris-Dauphine\Documents\GitHub\kaggle_challenge\kaggle_challenge\lib.py:156: RuntimeWarning: divide by zero encountered in log
  logit = np.log(logproba[:,1]/(1-logproba[:,1]))


Save data to relevant format

In [40]:
ctr = 'final'#int(input('Current submission attempt:\t'))
saveDataToFormattedSubmissionFile(logit, f'Data/WL_test_pred{ctr}.csv')

filename = 'Data/Models/lr_.sav'
pkl.dump(lr, open(filename, 'wb'))